# Deep First Search Algorithm

---


##### กำหนดปัญหา
- สมมุติว่าเราต้องการวางแผนการเดินทางไปยังหลายจุดหมายที่แตกต่างกัน ซึ่งสถานที่ต่างๆ เหล่านี้มีเส้นทางเชื่อมต่อกัน อาจจะมีการเลือกเส้นทางที่ต้องเดินทางไปเยี่ยมชมหลายแห่งในแต่ละครั้ง ทั้งที่บางเส้นทางอาจยาวและบางเส้นทางอาจสั้นกว่า
ในสถานการณ์เช่นนี้ เราต้องการหาวิธีการเลือกเส้นทางที่ สามารถเดินไปลึกสุด ก่อน (คือไปถึงจุดหมายที่เราตั้งใจจะไปให้เร็วที่สุดในเส้นทางนั้น) แล้วค่อยย้อนกลับมาและลองเส้นทางอื่นหากเส้นทางแรกไม่สำเร็จหรือถึงทางตัน ซึ่งตรงกับแนวคิดการใช้ DFS (Depth-First Search) ในการค้นหาผลลัพธ์ โดยเลือกเดินทางไปในเส้นทางที่ลึกที่สุดก่อนเสมอ


##### แก้ไขปัญหา
- การใช้ DFS (Depth-First Search) ในการแก้ปัญหาการวางแผนเที่ยวหลายสถานที่แบบลึกก่อน สามารถอธิบายได้ว่ามันเป็นการสำรวจเส้นทางจากจุดเริ่มต้นไปยังจุดหมาย โดยเลือกเส้นทางที่ลึกที่สุดก่อนและพยายามเดินทางไปให้สุดทางจนไม่สามารถไปต่อได้ จากนั้นหากพบทางตันจะย้อนกลับไปยังจุดที่เคยไปแล้วเพื่อสำรวจเส้นทางอื่น ๆ ที่ยังไม่เคยไป จนกว่าจะพบเส้นทางที่สามารถไปถึงจุดหมายได้หรือไม่สามารถหาทางไปได้เลย

## graph.py


In [1]:
# เส้นทางของ graph
# key คือ vertex
# value คือ vertex ที่เชื่อมต่อกันด้วยเส้น edge กับ vertex ของ key (neighbor)

# ทุก graphs ที่อยู่ใน module นี้เป็น Undirected Graphs ทั้งหมด
# กราฟอันที่ 1
graph = {
    "A": ["B", "C"],
    "B": ["A", "D"],
    "C": ["A", "D"],
    "D": ["B", "C", "E"],
    "E": ["D"],
}

# กราฟอันที่ 2
graph2 = {
    "A": ["B", "G"],
    "B": ["A", "G", "C"],
    "C": ["B", "E"],
    "E": ["C", "G", "J", "K"],
    "G": ["B", "E", "J"],
    "J": ["G", "E", "D", "F"],
    "D": ["J", "F", "K"],
    "F": ["J", "D", "K"],
    "K": ["E", "D", "F"],
}

# กราฟอันที่ 3 
# เป็นกราฟแบบ tree
graph3 = {
    "A": ["B", "C"],
    "B": ["A", "D", "E"],
    "D": ["B", "H", "I"],
    "H": ["D"],
    "I": ["D"],
    "E": ["B"],
    "C": ["F", "G"],
    "F": ["C"],
    "G": ["J", "C"],
    "J": ["K", "G"],
    "K": ["J"],
}

# รวมกราฟทั้งหมด
all_graphs = {"graph1": graph, "graph2": graph2, "graph3": graph3}

<p align="center">ภาพกราฟอันที่ 1</p>

<div align="center">
    <img src="https://raw.githubusercontent.com/WarinCode/algorithm-project/refs/heads/main/imgs/graph.png">
</div>

<br>

<p align="center">ภาพกราฟอันที่ 2</p>

<div align="center">
    <img src="https://raw.githubusercontent.com/WarinCode/algorithm-project/refs/heads/main/imgs/graph2.png">
</div>

<br>

<p align="center">ภาพกราฟอันที่ 3</p>

<div align="center">
    <img src="https://raw.githubusercontent.com/WarinCode/algorithm-project/refs/heads/main/imgs/graph3.png">
</div>

## main.py


In [2]:
# function สำหรับการเช็ค vertices ใน graph ว่าถ้ามีจะ return True ไม่มีจะ return False
def has_vertices(graph, start, goal):
    # เก็บเป็น set เพื่อให้ไม่มี vertices ที่มีค่าซ้ำกัน
    vertices = set()

    # วนลูปรอบแรกของ graph ให้ดึง key และ value ใน dict ออกมา
    for key, values in graph.items():
        # เพิ่มค่า key ลงใน all_vertices
        vertices.add(key)
        # วนลูปรอบที่สองของ values เพราะ value ที่ได้มาตอนแรกใน dict เป็น list
        # ต้องวน loop อีกรอบเพื่อได้ค่า element
        for value in values:
            # เพิ่มค่า value
            vertices.add(value)

    # เขียน condition return ออกมาว่าถ้า start และ goal อยู่ใน all_vertices
    # ให้ return True ไม่อยู่ให้ return False
    return start in vertices and goal in vertices

In [3]:
# Algorithm: Deep-Firth-Search
def dfs(graph, start, goal, path=None):
    # เช็คว่าถ้าค่า path เป็น None ให้ทำเงื่อนไขต่อไปนี้ (สำหรับรัน algorithm นี้ในครั้งแรก)
    if path is None:
        # กำหนดค่า path ให้เป็น list
        path = []
        # เช็ค condition นี้ว่าถ้า vertices ที่รับค่ามาจาก paramters ไม่มีอยู่ใน graph ให้ throw exception นี้ออกไป
        if not has_vertices(graph, start, goal):
            raise Exception(f"ไม่มี vertices {start} หรือ {goal} ที่อยู่ใน graph!")
        
    # เพิ่มค่า start ลงใน path
    path.append(start)

    # ถ้าพบเส้นทางเป้าหมายแล้วให้ return ค่า path ออกมา
    if start == goal:
        return path

    # วน loop หาเส้นทางของ goal เมื่อวน loop ค่าของตัวแปร neighbor จะเป็นค่า value ของ dict
    for neighbor in graph[start]:
        # ถ้าค่า neighbor ไม่อยู่ใน path ให้เรียกใช้ dfs ตัวมันเองอีกครั้ง(recursive)
        # และ หลีกเลี่ยงปัญหาเกิดการเก็บค่าเดิมของ path ที่เราเคยเดินมาแล้ว
        if neighbor not in path:
            # ส่ง arguments ไปใหม่เปลี่ยนจาก start -> neightbor ที่เหลือเหมือนเดิม
            result = dfs(graph, neighbor, goal, path.copy())
            # เช็คถ้ามีผลลัพธ์(มี elemnet ใน list) ให้ return ผลลัพธ์นั้นออกมา
            if result:
                return result  # Return the first found path

In [4]:
# กำหนดค่า start และ goal
# start คือจุดเริ่มต้นใน graph

# สำหรับรับค่าข้อมูลผ่าน console
# start = input("start : ").upper().strip()
# goal = input("goal : ").upper().strip()

# สำหรับกำหนดค่าแบบปกติ
start = "A"
# goal คือจุดเป้าหมายที่ต้องไปให้ถึงใน graph
goal = "I"

# เขียน try catch เพื่อดักจับอาจจะมีการเกิด exception จาก dfs นี้ไว้
try:
    # เรียกใช้ algorithm: dfs เพื่อหาเส้นทาง
    path = dfs(all_graphs["graph3"], start, goal)
    # แสดงผลลัพธ์
    print(f"เส้นทางจาก {start} ไปยัง {goal} คือ")
    print(" -> ".join(path))
except Exception as e:
    # แสดงข้อความของ exception
    print(e.__str__())

เส้นทางจาก A ไปยัง I คือ
A -> B -> D -> I
